In [ ]:
#This Supernova Cosmology project program
#Free for all kinds of use, change and redistribution
#Copyleft (Though I don't really know what it means)
#Created by Shibo Chen, October 31, 2016 @ University of Michigan
#Multi-contributors as shown in the list below

In [ ]:
%matplotlib inline
from pylab import *
import numpy

In [31]:
#I don't know much about jupyter so let's use it as a web-based python :)
#first, let's create a class

class Supernova():
    
    #init the supernova
    def __init__(self,redshift,magnitude,errinMagnitude):
        self.redshift=redshift
        self.magnitude=magnitude
        self.errinMagnitude=errinMagnitude
        
    #This function will return the computed magnitude.
    #if you don't give Omage DE, it will assume it is a flat universe
    #And I really hate python, why I can't declare something as const
    def computedMagnitude(self,omegaMatter,omegaDarkEnergy=None):
        if omegaDarkEnergy==None:
            omegaDarkEnergy=1-omegaMatter
    
    #I don't know what is going on here on Jupyter so I created a getter method explicitly
    def getAttribute(self,attr):
        if attr=="redshift":
            return self.redshift
        elif attr=="magnitude":
            return self.magnitude;
        elif attr=="errinMagnitude":
            return self.errinMagnitude
        else:
            return "no attribute "+attr+" found in supernova"
        
        

#Then we create a empty list to store supernovas
supernovas=[]

#Open the file 
dataFile=open('SupernovaData','r')

#read in the data line by line and push the supernovas into our list
for line in dataFile:
    datas=line.split()
    
    redshift=float(datas[1])
    magnitude=float(datas[2])
    errinMagnitude=float(datas[3])
    
    supernova=Supernova(redshift,magnitude,errinMagnitude)
    supernovas.append(supernova)
    
dataFile.close()

In [30]:

likelihood=0.0
chisquared=0.0
standardMagnitude=0.0

0.028488
